In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("data/external_data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 59 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   numer_sta  3322 non-null   int64  
 1   date       3322 non-null   object 
 2   pmer       3322 non-null   int64  
 3   tend       3322 non-null   int64  
 4   cod_tend   3322 non-null   int64  
 5   dd         3322 non-null   int64  
 6   ff         3322 non-null   float64
 7   t          3322 non-null   float64
 8   td         3322 non-null   float64
 9   u          3322 non-null   int64  
 10  vv         3322 non-null   int64  
 11  ww         3322 non-null   int64  
 12  w1         3315 non-null   float64
 13  w2         3312 non-null   float64
 14  n          3166 non-null   float64
 15  nbas       3317 non-null   float64
 16  hbas       2869 non-null   float64
 17  cl         2909 non-null   float64
 18  cm         1941 non-null   float64
 19  ch         1678 non-null   float64
 20  pres    

Possible Relevant Features for Bike Count Prediction:

1. Temperature (t): People are more likely to bike in comfortable temperatures.
2. Precipitation (rr1, rr3, rr6 etc.): Rain or snow can deter biking.
3. Wind Speed (ff, raf10, rafper): Strong winds might affect biking.
4. Humidity (u): High humidity can make conditions uncomfortable for biking.
5. Visibility (vv): Poor visibility could reduce biking for safety reasons.
6. Cloud Cover (n, cl, cm, ch): May indirectly affect biking decisions.
7. Date and Time: To align with hourly bike count data.

In [4]:
df["date"] = pd.to_datetime(df["date"])

In [5]:
df.columns

Index(['numer_sta', 'date', 'pmer', 'tend', 'cod_tend', 'dd', 'ff', 't', 'td',
       'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas', 'cl', 'cm', 'ch',
       'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24',
       'tminsol', 'sw', 'tw', 'raf10', 'rafper', 'per', 'etat_sol', 'ht_neige',
       'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24', 'phenspe1',
       'phenspe2', 'phenspe3', 'phenspe4', 'nnuage1', 'ctype1', 'hnuage1',
       'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3', 'hnuage3',
       'nnuage4', 'ctype4', 'hnuage4'],
      dtype='object')

In [6]:
df = df.drop(
    [
        "numer_sta",
        "niv_bar",
        "geop",
        "tn12",
        "tn24",
        "tx12",
        "tx24",
        "tminsol",
        "sw",
        "tw",
        "phenspe1",
        "phenspe2",
        "phenspe3",
        "phenspe4",
        "per",
    ],
    axis=1,
)

In [7]:
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace=True)

In [8]:
df.drop(
    columns=[
        "hnuage2",
        "nnuage2",
        "ctype2",
        "hnuage3",
        "hnuage4",
        "ctype3",
        "ctype4",
        "nnuage3",
        "nnuage4",
    ],
    inplace=True,
)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3322 entries, 2021-01-01 00:00:00 to 2020-09-30 21:00:00
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pmer       3322 non-null   int64  
 1   tend       3322 non-null   int64  
 2   cod_tend   3322 non-null   int64  
 3   dd         3322 non-null   int64  
 4   ff         3322 non-null   float64
 5   t          3322 non-null   float64
 6   td         3322 non-null   float64
 7   u          3322 non-null   int64  
 8   vv         3322 non-null   int64  
 9   ww         3322 non-null   int64  
 10  w1         3315 non-null   float64
 11  w2         3312 non-null   float64
 12  n          3166 non-null   float64
 13  nbas       3317 non-null   float64
 14  hbas       2869 non-null   float64
 15  cl         2909 non-null   float64
 16  cm         1941 non-null   float64
 17  ch         1678 non-null   float64
 18  pres       3322 non-null   int64  
 19  tend24     3

w1, w2: replace nan with 3

n: NaN replace with 0

n: replace the 101. with 100.

nbas: replace nan with the mode of the day

hbas: replace with mode of the day

cl, cm, ch: replace with mode of the day

tend24: replace nan with mean of the day

raf10: replace with mean of the day

etat_sol: replace with mode of the day

ht_neige: replace negative with 0 and nan with mode of the day

ssfrai: replace nan with mode of the day 

persfrai: drop column

rr1 to rr24: replace nan with mean of the day

hnuage1: replace nan with mean of the day

nnuage1: replace nan with mode of the day

In [10]:
df["w1"].fillna(3, inplace=True)
df["w2"].fillna(3, inplace=True)
df["n"].fillna(0, inplace=True)

In [11]:
df["n"].replace(101.0, 100.0, inplace=True)

In [12]:
def replace_with_mode(df, column):
    # Calculate the mode of the day
    mode_of_day = df.groupby(df.index.date)[column].transform(
        lambda x: x.mode()[0] if not x.mode().empty else x
    )

    # Replace NaNs with the mode of the day
    df[column].fillna(mode_of_day, inplace=True)

    # Calculate the mode of the week
    mode_of_week = df.groupby(df.index.isocalendar().week)[column].transform(
        lambda x: x.mode()[0] if not x.mode().empty else x
    )

    # Replace remaining NaNs with the mode of the week
    df[column].fillna(mode_of_week, inplace=True)

In [13]:
for column in [
    "nbas",
    "hbas",
    "cl",
    "cm",
    "ch",
    "etat_sol",
    "ssfrai",
    "nnuage1",
    "ctype1",
]:
    replace_with_mode(df, column)

In [14]:
df["ht_neige"] = df["ht_neige"].apply(lambda x: max(x, 0))

In [15]:
replace_with_mode(df, "ht_neige")

In [16]:
def replace_with_mean(df, column):
    # Calculate the mean of the day
    mean_of_day = df.groupby(df.index.date)[column].transform("mean")

    # Replace NaNs with the mean of the day
    df[column].fillna(mean_of_day, inplace=True)

    # Calculate the mean of the week
    mean_of_week = df.groupby(df.index.isocalendar().week)[
        column].transform("mean")

    # Replace remaining NaNs with the mean of the week
    df[column].fillna(mean_of_week, inplace=True)

In [17]:
for column in ["tend24", "raf10", "rr1", "rr3", "rr6", "rr12", "rr24", "hnuage1"]:
    replace_with_mean(df, column)

In [18]:
df.drop(columns=["perssfrai"], inplace=True)

In [19]:
df.head()

,pmer,tend,cod_tend,dd,ff,t,td,u,vv,ww,...,ht_neige,ssfrai,rr1,rr3,rr6,rr12,rr24,nnuage1,ctype1,hnuage1
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,990,2,...,0.00,0.00,0.0,0.0,0.0,0.0,2.0,1.0,6.0,600.0
2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,210,40,...,0.00,0.00,0.0,0.0,0.0,0.0,1.2,1.0,6.0,1500.0
2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,3660,3,...,0.00,0.00,0.0,0.0,0.0,0.0,1.0,3.0,6.0,480.0
2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,3500,10,...,0.01,0.01,0.0,0.2,0.2,0.2,0.2,1.0,6.0,1740.0
2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,8000,2,...,0.00,0.00,0.0,0.0,0.2,0.2,0.2,1.0,8.0,330.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3321 entries, 2021-01-01 00:00:00 to 2020-09-30 21:00:00
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pmer      3321 non-null   int64  
 1   tend      3321 non-null   int64  
 2   cod_tend  3321 non-null   int64  
 3   dd        3321 non-null   int64  
 4   ff        3321 non-null   float64
 5   t         3321 non-null   float64
 6   td        3321 non-null   float64
 7   u         3321 non-null   int64  
 8   vv        3321 non-null   int64  
 9   ww        3321 non-null   int64  
 10  w1        3321 non-null   float64
 11  w2        3321 non-null   float64
 12  n         3321 non-null   float64
 13  nbas      3321 non-null   float64
 14  hbas      3321 non-null   float64
 15  cl        3321 non-null   float64
 16  cm        3321 non-null   float64
 17  ch        3321 non-null   float64
 18  pres      3321 non-null   int64  
 19  tend24    3321 non-null   float64

In [21]:
df = df.drop_duplicates()

In [23]:
df.to_csv("data/external_data_cleaned_updated.csv")

# Everything below is old

Filling NaN values in continuous columns by mean of the day

In [ ]:
df_relevant["day"] = df_relevant["date"].dt.date

In [ ]:
for col in relevant_columns:
    daily_avg = df_relevant.groupby("day")[col].transform("mean")
    df_relevant[col].fillna(daily_avg, inplace=True)

In [ ]:
df_relevant.head()

In [ ]:
df_relevant.info()

In [ ]:
df_relevant["week"] = df_relevant["date"].dt.isocalendar().week

In [ ]:
# Handling the remaining NaN values with mean and mode

for col in relevant_columns:
    weekly_avg = df_relevant.groupby("week")[col].transform("mean")
    df_relevant[col].fillna(weekly_avg, inplace=True)

In [ ]:
df_relevant["precipitation"] = (df_relevant["rr1"] > 0).astype(int)
df_relevant["cloudy_day"] = (df_relevant["cl"] > 50).astype(int)

In [ ]:
for column in ["rr1", "rr3", "rr6"]:
    df_relevant.loc[df_relevant[column] < 0, column] = 0

In [ ]:
df_relevant = df_relevant.drop_duplicates()

In [ ]:
# Saving the cleaned DataFrame to a CSV file
df_relevant.to_csv("data/external_data_cleaned.csv", index=False)